In [ ]:
!pip install fastapi
!pip install fastapi uvicorn
!pip install colabcode
!pip install "uvicorn[standard]"
!pip install python-multipart
! ngrok config add-authtoken 2i45sJc1zS7L3kU0d6yb0wfmPAE_27rCfiAQGkSPutoUhBUeq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import uvicorn
from typing import Union
from colabcode import ColabCode
from google.colab import drive
import shutil
from pyngrok import ngrok
from fastapi import FastAPI, BackgroundTasks, File, UploadFile
from fastapi.responses import StreamingResponse
from typing import List
import numpy as np
import tensorflow as tf
import cv2
import json
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/convlstm-model'
target_frames = 30
resize = 224

In [ ]:
cc = ColabCode(port=8000, code=False)
app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

In [ ]:
@app.on_event("startup")

# Function to load the model
def model_loading():
    global model
    model_path = '/content/drive/MyDrive/convlstm-model'  # Adjust the path to your TensorFlow model
    tf.saved_model.load(model_path)
    print(f"Model loaded from {model_path}")

<ipython-input-6-090bad9c9258>:1: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
def verify_model_loading(model_path):
    model = tf.saved_model.load(model_path)
    print(f"Model loaded from {model_path}")
    return model

In [ ]:
class FrameData(BaseModel):
    data: List[List[float]]

class FramesList(BaseModel):
    frames: List[FrameData]

@app.post("/")
def process_data(framesList: FramesList):
  frames = []
  try:
    print("Received framesListttttttt")
    for frame in framesList.frames:
        frame_data = np.array(frame.data, dtype=np.uint8)
        print("Received frame data shape: ", frame_data.shape)
        frame_data = cv2.resize(frame_data, (resize, resize), interpolation=cv2.INTER_AREA)
        frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
        frames.append(frame_data)
    frames = np.array(frames)
    frames_diff_input = np.diff(frames, axis=0).astype(np.float32)  # Compute differences
    frames_input = tf.convert_to_tensor(frames[np.newaxis, ...])
    frames_diff_input = tf.convert_to_tensor(frames_diff_input[np.newaxis, ...])
    model = tf.keras.models.load_model(model_path)
    prediction = model([frames_input, frames_diff_input], training=False)
    prediction = np.squeeze(prediction)

    is_violent = prediction >= 0.50
    print(f"Is violent: {is_violent}")
    print(prediction)
  except Exception as e:
    print("Error: ", e)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [332]
INFO:uvicorn.error:Started server process [332]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.


Public URL: https://a220-34-69-128-235.ngrok-free.app


INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.


Model loaded from /content/drive/MyDrive/convlstm-model


INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Received framesListttttttt
Error:  setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
INFO:     41.233.164.38:0 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [332]
INFO:uvicorn.error:Finished server process [332]
